In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

batch_size = 2
src_len = torch.Tensor([2, 4]).to(torch.int32)
tgt_len = torch.Tensor([4, 3]).to(torch.int32)

# Step1: 单词索引构成源句子和目标句子，构建patch，并且做了padding，默认值为0

# Step2：构造word embedding

# Step3：构造positional encoding

# Step4：构造encoder的self-attention mask
# mask的shape：[batch_size, max_src_len, max_src_len]，值为1或-inf

# Step5：构造intra-attention的mask
# Q @ K^T shape: [batch_size, tgt_seq_len, src_seq_len]
vaild_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(src_len)-L)), 0) for L in src_len], dim=0), 2) # 为什么最后一个参数为2？因为矩阵运算是不会让batch_size这个维度参与的，只会让seq_len这个维度参与，因此需要对非0维剩下的维度进行扩维
vaild_decoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(tgt_len) - L)), 0) for L in tgt_len], dim=0), 2)


vaild_cross_pos_matrix = torch.bmm(vaild_decoder_pos, vaild_encoder_pos.transpose(1, 2)) # Q @ K^T
invaild_cross_pos_matrix = 1 - vaild_cross_pos_matrix

mask_cross_attention = invaild_cross_pos_matrix.to(torch.bool)
# # print(vaild_encoder_pos) # '1'表示真实的token，而'0'表示是由padding来的
# print("The shape of vaild_encoder_loss is: ", vaild_encoder_pos.shape)

# print(vaild_decoder_pos)
# print("The shape of vaild_decoder_pos is : ", vaild_decoder_pos.shape)

# print(vaild_cross_pos_matrix)
# print("The shape of vaild_cross_pos is : ", vaild_cross_pos_matrix.shape)

# print(invaild_cross_pos_matrix)

# print(mask_cross_attention)
# print("The shape of mask_cross_attention is : ", mask_cross_attention.shape)

# Step 6：构造decoder self-attention的mask
tri_matrix = [torch.tril(torch.ones((L, L))) for L in tgt_len]

print(tri_matrix)
# https://www.bilibili.com/video/BV1Qg411N74v?t=2009.9



[tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]]), tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])]
